# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [1]:
# import all libraries needed for this task
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Mount from google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Open the holbrook text file. 
sentences = open('/content/gdrive/My Drive/holbrook.txt').readlines()

# Parsing the sentences to original and corrected.
list_original=[]
list_corrected=[]
list_indexes=[]

for i in sentences:
    words=word_tokenize(i)
    ind=0
    original_data=[]
    corrected_data=[]
    indexes=[]
    
    for word in words:
        if(re.match(r'\w+[|](\w+)',word)):
            
            # Next word beside '\w+[|](\w+)' is stored in the corrected_data list.
            corrected_data.append(re.split(r'\w+[|](\w+)',word)[1])
            # The word previous to '\w+[|](\w+)' is stored in the original_data list.
            original_data.append(re.split(r'(\w+)[|]\w+',word)[1])
            # The index of the error in question        
            indexes.append(ind)
            
        else:
           
            # If there is no '\w+[|](\w+)' in sentence, sentence is stored in list_original and list_corrected.
            original_data.append(word)
            corrected_data.append(word)
            
        # index increases by +1 when the following if/else statement is executed.
        ind+=1
    
    # the data is updated and stored in the relavant list that it's associated with.   
    list_original.append(original_data)
    list_corrected.append(corrected_data)
    list_indexes.append(indexes)
    
    


In [0]:
# Loading the list_original, list_corrected and list_indexes data to 'sentences'.
data=[]
for i in range(len(sentences)):
    data.append({"original":list_original[i],"corrected":list_corrected[i],"indexes":list_indexes[i]})

In [0]:
# Testing
# Prints list of all the sentences
sentences

In [0]:
# Testing.
# Printing the data in sentence numner 2
print(data[2])


# Test your code with the following
assert(data[2] == {
 'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'],
 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'],
 'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [0]:
# Splitting the data to test 100 lines and remaining training lines
test = data[:100]
train = data[100:]

In [0]:
# Splitting the data to test - first 100 lines and remaining training lines
def test_train_split():

  # Splitting the data to test - first 100 lines and remaining training lines.
    test = data[:100]
    train = data[100:]
    
    # Seperating the train original, test original, test corrected and train corrected from dictionary to list.
    train_corrected = [elem['corrected'] for elem in train]
    tokenizer = RegexpTokenizer(r'\w+')
    test_corrected = [elem['corrected'] for elem in test]
    test_original = [elem['original'] for elem in test]
    
    # Removing all special characters from the list.
    test_original = [tokenizer.tokenize(" ".join(elem)) for elem in test_original]
    test_corrected = [tokenizer.tokenize(" ".join(elem)) for elem in test_corrected]
    train_corrected = [tokenizer.tokenize(" ".join(elem)) for elem in train_corrected]
    
    # Returns the following
    return test_corrected, test_original, train_corrected

# Test and Training data.
test_corrected, test_original, train_corrected = test_train_split()

In [0]:
# Testing the test_corrected data to see if it prints out correctly
test_corrected

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [7]:
# Import relevant libraries
from collections import Counter

# from sklearn.feature_extraction.text import CountVectorizer
# The following function returns a unigram function for a given word in the train data
def create_unigrams(train):
    text=''
    for i in range(0,len(train)):
        corrected=train[i]['corrected']
        for j in corrected:
            text+=j+' '
            
    # split the texts into tokens
    unigrams = word_tokenize(text)
    unigrams = [token.lower() for token in unigrams if len(token) > 1] 
    
    # the conunter is used to implement the number of occurences of all words and it may be called many times
    # Returns the unigram and the number of occurrences of the unigram
    unigram_freq=Counter(unigrams)
    return unigrams,unigram_freq

# Defines the function unigram for each word
# Returns the frequency of all words and thei unigram probability from the corrected training sentences  
def unigram(word):
    return prob(unigram_frequency,word)

# Defines the function prob for each unigram and its associated word
# Returns the actual unigram frequency along with its associated word
def prob(unigram_freq,word):
    return unigram_freq[word]

unigrams,unigram_frequency=create_unigrams(train)
print(unigram('me'))



#Test your code with the following
assert(unigram("me")==87)

87


## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [8]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [9]:
import nltk
def get_candidates(token):
  
    #Collects the set of all unique tokens in train, 'unigrams' below is the list of all of the tokens. unq_unigrams sorts them in a list by uniqueness
    unq_unigrams=list(set(unigrams))
    similar_text=[]
    
    #Calculate the minimal edit distance, that is the lowest value for the function edit_distance between token and a word in train(unq_unigrams)
    for i in unq_unigrams:
        if(nltk.edit_distance(i,token)<2):
            similar_text.append(i)
            similar_text.sort(reverse=True)
    
    #Output all unique words in train(unq_unigrams) that have this same (minimal) edit_distance value
    return similar_text

print(get_candidates("minde"))

# Test your code as follows
assert get_candidates("minde") == ['mine', 'mind']

['mine', 'mind']


## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [10]:
# This function returns the corrected sentence based on unigram probability
def correct(sentence):
    corrected_sentences=[]
    
    #For finding candidates with minimal edit distance
    for i in sentence:
      
      # Calls the get_candidates function to choose a word in the dictionary that has minimal edit distance and has the highest unigram probability
      # Calls edit_dist and unigram_prob also
        alternatives=get_candidates(i)
        edit_dist=[]
        unigram_prob=[]
        
        # If the word has an alternative of greater than 0 then run the following function
        if(len(alternatives)>0):
          
            # For each word in alternatives(get_candidates) calculate the minimal edit distance and the highest unigram probability
            for j in alternatives:
                unigram_prob.append(unigram(j))
                edit_dist.append(nltk.edit_distance(i,j))
                
                # The following two lines of code converts the words with the minimal edit distance and has the highest unigram probability and lists them in a dictionary.
                # zip returns a list while dict is the constructor
            alternative_dist=dict(zip(alternatives,edit_dist))    
            alternative_unigram=dict(zip(alternatives,unigram_prob))
            
            # Sorts the previous two lines of code with respect to the second element. key=lambda x[1] does this
            alternative_dist_s=sorted(alternative_dist.items(), key=lambda x: x[1])
            alternative_unigram_s=sorted(alternative_unigram.items(),key=lambda x: x[1],reverse=True)
            
            # Take the suggested word with minimum probability 
            y=min(alternative_dist_s,key=lambda item:item[1])
            unig_out=[item for item in alternative_dist_s if y[1] in item]
            if(len(unig_out)==1):
                corrected_sentences.append(unig_out[0][0])
            else:
              
                # Take the suggested word with maximum probability
                y=max(alternative_unigram_s,key=lambda item:item[1])
                unig_out=[item for item in alternative_unigram_s if y[1] in item]
                corrected_sentences.append(unig_out[0][0])
                
    # Return the corrected sentence
    return corrected_sentences

# Test code as follows
print(correct(["this","whitr","cat"]))
assert(correct(["this","whitr","cat"])==["this","white","cat"])

['this', 'white', 'cat']


##Task 5 (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [11]:
def accuracy(corrected, original):
    
    #The following two lines of code is associted with word to word accuracy calculation. It compares each individual word (original_corrected) with the actual word (corrected_l) and calculates accuracy.
    corrected_l = corrected
    original_corrected = correct(original)
    
    # if the length of the individual word in original_corrected  is the same as the ouputed corrected_l then there has been no change so the prediction is accuracte, therefore taking accuracy as+1.
    if len(original_corrected) == 0 and len(corrected_l) == 0:
        accuracy = 1.0
    else:
      # The following takes all of the original_corrected words that are the same as the corrected_l words divides it by the length of the corrected_l list. Thus, giving the accuracy.
      accuracy = sum(1 for x,y in zip(original_corrected,corrected_l) if x == y) / len(set(corrected_l))
    
    return accuracy
  
acc = []
for i in tqdm(range(len(test_corrected))):
    acc.append(accuracy(test_corrected[i], test_original[i]))

        
print("Average Accuracy of words in each sentence: ", round(sum(acc)/len(acc)*100), "%")

100%|██████████| 100/100 [01:15<00:00,  1.22it/s]

************************************************EVALUATION**********************************************************
Average Accuracy of words in each sentence:  44 %
12 out of 100 sentences predicted correctly without any error.
Elapsed Time is:  75.95444583892822


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lecutures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)

...............................................................................................................................................................

##**Modification Ideas and Implementation:**

** Wordnet.words():**
*	This can be implemented in order to check if the word in question is a valid word in the text corpus. It’s the most reliable way to measure the semantic similarity of two words. It’s an NLTK corpus reader that can be imported by the following line of code:

>>> from nltk.corpus import wordnet


** Lemmatisation and Stemming:**
* These text normalisation techniques are widely used in natural language processing to prepare text, words and documents for further processing. The NLTK library contains the necessary methods to conduct lemmatisation and stemming on the training data. Stemming is the process of reducing inflection in words to their root forms. It involves linking a certain word or words to the derived stem. Stems are generated by adding or removing suffixes/prefixes such as –ed, -ize, -s, -de and mis with a word. I would add these suffixes/prefixes to the word in question to see if the word is then present in the dictionary. This would lead to the careful, precise preparation of documents for processing which can lead to increased accuracy in the spellchecking system.  

** Detect tense of the words in the training data:**
*	The main outcome of this is to detect the tense of the words in the training data. Firstly, the POS tags need to be obtained. According to the part-of-speech tags in the Penn treebank project, VBD and VBN refer to the past tense. VBD and VBN also refer to the past participle of a verb. A parser may also be used in order to identify the verb phrase. 
*	"By the time I will reach there, he'd have already left”
*	Sentences such as the following may cause confusion and certain rules regarding POS tags must be adhered to. The auxiliary verb in the above sentence is followed by a past tense verb and is this presented as the present perfect tense.

** Word derivative - most probable word:**
*	The nltk_derivationally_related_forms can be implemented here. The get_candidates function can benefit greatly from this implementation as every suggested word from it can be identified, its unigram probability can then be taken to identify the most relatable word to replace the incorrect word in the sentence. It's based on language model probability and can greatly increase the accuracy of the spellchecker.

** Smoothing:**
* Smoothing is another feature than can be implemented to increase the accuracy of your spellchecker. Its objective is to basically increase the probability of matching misspelled/incorrect words with its derivative form. It greatens their probability towards unknown n-grams. Upon doing some research in deciding what's the best smoothing implementation to improve the accuracy, the interpolated modified Kneser-Ney smoothing yields the best performance. There are three ways that produce the benefits of smoothing: absolute-discounting, interpolation and word histories. Aiming to predict the probability of a word in a given context is very difficult but these three techniques aim to simplify this process leading to a more precise, accurate spellchecker. 

** Fuzzy Ratio:**
* Fuzzy ratio can also be implemented for string matching and corresponding changes can be made. Fuzzy String Matching is the process of identifying strings that are similar to another string based on the similarity of their given pattern. The similarity is measured by edit distance. Edit distance is the amount of basic operations needed to convert a string into its exact/appropriate match. These basic operations include insertion of a particular character, deletion of a particular character and substitution of a particular character. Fuzzy String Matching can be implemented excellently for spellchecking NLP systems while greatly increasing accuracy ensuring an efficient model.

** Example:** 
* > from fuzzywuzzy import fuzz
* > fuzz.ratio("this is a test", "this is a test!")
* > 96

** Recurrent Neural Network:**
* Implementing a recurrent neural network based language model could improve the accuracy of the algorithm developed in Task 3 and 4. A neural network can order and score sentences based on the likelihood that the sentence is a valid/accurate prediction. Grammatical and semantic correctness is the main outcome here. If scoring and predicting the sentence isn’t sufficient enough then a new sentence/new piece of text can be generated. 
* The key idea in recurrent neural networks is to accurately predict the next word in the sentence. In order to do this it’s essential to know what word comes before it in the sentence in question. RNNs are called recurrent for this very reason as the neural network performs the same task on each element (word) in the sequence (sentence). The output is dependent on the previous computations. RNNs also have the ability to store and calculate valuable information that has been captured thus far. The ‘brain’ of the RNN is knowledgeable and excellent. 
* I would do the following to implement a recurrent neural network into my model. Lets say there is 7 words in a sentence. The RNN would compute this sentence into a 7 layer neural network. Each word in the sentence has its own subsequent layer. The formula that’s associated with RNNs Is then applied to each of the 7 layers of the sentence. 

** LSTM: Long short-term memory networks:**
* Vanishing gradients and exploding gradients are two major problems in traditional RNNs. The LSTM model aims to resolve these issues by implementing a framework called ‘memory cell’. The memory cell consists of four core elements: an input gate, a self-recurrent connection (a connection to itself), a forget gate and an output gate. These elements act as a translator and modifies interactions between the memory cell and its environment. LSTM networks can greatly improve the accuracy of the algorithm developed in task 3 and 4.  If i were to implement this I wouldo exactly as I have just explained: implement a LSTM model so that a memory cell can create the four gates mentioned above. Each word from the text file can then be processed through the network leading to accurate predictions, increasing accuracy.

* The gates contain the ability to:
* **Input:** alter the state of the memory cell
* **Output:** prevent the effect the memory cell has on specific neurons
* **Forget:** refrain the memory cells recurrent connection while remembering past connections inside



## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [0]:
corrected_l=[]
original_corrected=[]

# Function calculating the improved accuracy of the algorithm developed in part 5. 
def accuracy(corrected_l, original_corrected):
    
    # Corrects the relevant list to convert to lower for efficiency puposes of the spellchecker.
    for i in test:
        c=0
        for words in i['original']:
            if(c in i['indexes']):
                original_corrected.append(correct(words.lower()))
            else:
                original_corrected.append(words.lower())
            c+=1
        for words in i['corrected']:
            corrected_l.append(words.lower())
    
    # The following takes all of the original_corrected words that are the same as the corrected_l words divides it by the length of the corrected_l list. Thus, giving the accuracy.
    accuracy_t=sum(1 for x,y in zip(original_corrected,corrected_l) if x == y) / len(corrected_l)
    
    # Returns the result of the above accuracy equation.
    return accuracy_t

acc=[]
for i in tqdm(range(len(test_corrected))):
    acc.append(accuracy(test_corrected[i], test_original[i]))

    # Prints average accuracy of words in each sentence
print("Average Accuracy of words in each sentence: ", round(sum(acc)/len(acc)*100), "%")
